# Chap 6. 다양한 SNS리뷰 정보 수집하기_인스타그램_유투브 

In [51]:
import time 
txt = '서진수 꽃미남'
for i in txt:
    print(i)
    time.sleep(0.5)

서
진
수
 
꽃
미
남


In [45]:
##########################################################################
# 인스타 그램의 해쉬태그 수집하기 - by 서진수
##########################################################################
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import os
import random
import unicodedata   # 인스타그램의 해시태그 수집 중 자음/모음 분리현상 방지용 모듈

#Step 2. 사용자에게 필요한 정보들을를 입력 받습니다.
print("=" *70)
print("   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다")
print("   본 제품은 서진수가 교육용으로 특별 제작했으며 ")
print("   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다")
print("   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^")
print("=" *70)

v_id = input("1.인스타그램의 ID를 입력하세요: ")
v_passwd = input("2.인스타그램의 비밀번호를 입력하세요: ")
query_txt = input("3.검색할 해쉬태그를 입력하세요(예: 강남맛집): ")
cnt = int(input('4.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 10)

f_dir=input('5.파일이 저장될 경로만 쓰세요(기본경로 : c:\\temp\\ ) : ')
if f_dir =='' :
    f_dir = "c:\\temp\\"

#Step 3. 결과를 저장할 폴더명과 파일명을 설정하고 폴더를 생성합니다.
s_time = time.time( )
query_txt2 = '인스타그램'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

# Step 4. 인스타그램 자동 로그인 하기
chrome_path = "c:/temp/chromedriver_91/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.instagram.com/")
driver.maximize_window()
time.sleep(random.randrange(1,5))

print("\n")
print("요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^")
print("\n")

#ID와 비번 입력후 로그인하기
eid = driver.find_element_by_name('username')
for a in v_id :
        eid.send_keys(a)
        time.sleep(0.3)
epwd = driver.find_element_by_name('password')
for b in v_passwd :
        epwd.send_keys(b)
        time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(5)

# Step 5. 검색할 해쉬태그 입력하기
#로그인 정보 나중에 저장하기
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()

time.sleep(1)

# 검색할 키워드 입력하기
element = driver.find_element_by_xpath('''//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input''')

for c in query_txt :
    element.send_keys(c)
    time.sleep(0.2)

time.sleep(5)
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a').click()

import pandas as pd

# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(10)

print('요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^')
item=[]     # 인스타그램 URL 주소 저장할 리스트
item2=[]
    
a = 1
while (a <= real_cnt):
    scroll_down(driver) 

    # Step 6. 전체 게시물의 원본 URL 추출하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    all = soup.find('article','KC1QD').find_all('a')

    for i in all:    
        url = i['href']
        item.append(url)          
        item2 = pd.Series(item).drop_duplicates()
    
        if len(item2) >= cnt :
            break
    a += 1
    print('요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^')
    
    if len(item2) >= cnt :
        break
   
# 추출된 URL 사용하여 전체 URL 완성하기
full_url=[]
url_cnt = 0
for x in range(0,len(item2)) :
    url = 'https://www.instagram.com' +item[x]
    full_url.append(url)
    url_cnt += 1
    
    if url_cnt > cnt:
        break

# 아래 for 반복문은 수집된 URL 주소를 확인하는 코드임        
#for c in range(1,len(full_url)+1) :
#    print(c,':',full_url[c-1])

#Step 7. 각 페이지별로 그림과 해쉬태그를 수집하기
count = 1        # 추출 데이터 건수 세기
hash_txt = []    # 해쉬 태그 저장 

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
import sys
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

count = 0

os.makedirs(f_dir+s+'-'+query_txt2+'-'+query_txt)
f_name=f_dir+s+'-'+query_txt2+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'

for c in range(0,len(full_url)) :
    driver.get(full_url[c])
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    f = open(f_name, 'a',encoding='UTF-8')

    # 해당 페이지의 해시태그 수집
    tags = soup.find('div','EtaWk')

    try :
        tags_1 = tags.find_all('a')
    except :
        pass
    else :
        for d in range(0, len(tags_1)) :
            tags = tags_1[d].get_text()
            tags_11 = tags.translate(bmp_map)
            tags_2 = unicodedata.normalize('NFC', tags_11)

            for i in tags_2 :
                if i[0:1]=='#' :
                    hash_txt.append(tags_2)
                    print(tags_2)
                    f.write("\n" + str(tags_2))
    f.close()
    count += 1
    
#Step 7. 요약 정보 출력하기    
e_time = time.time( )
t_time = e_time - s_time

print("=" *100)
print("총 소요시간: %s 초" %round(t_time,1))
print("총 저장 건수: %s 건 " %count)
print("파일 저장 경로: %s" %f_name)
print("=" *100)

driver.close( )

   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다
   본 제품은 서진수가 교육용으로 특별 제작했으며 
   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다
   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^
1.인스타그램의 ID를 입력하세요: 01095937708
2.인스타그램의 비밀번호를 입력하세요: gusgh202!!
3.검색할 해쉬태그를 입력하세요(예: 강남맛집): 강남맛집
4.크롤링 할 건수는 몇건입니까?: 20
5.파일이 저장될 경로만 쓰세요(기본경로 : c:\temp\ ) : c:\temp\


요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^


요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^
요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^
#카와카츠오토코
#합정맛집
#홍대맛집
#연남동맛집
#상수맛집
#망원동맛집
#서울맛집
#코밥심_여의도
#여의도의축복
#아루히
#아루히스시
#진주집
#피양옥여의도점
#강남스테이크
#서울맛집
#서울맛집추천
#서울맛집투어
#서울맛집베스트
#강남맛집
#강남맛집추천
#강남데이트
#강남데이트코스
#강남데이트코스추천
#강남레스토랑
#강남역맛집
#강남엿맛집베스트10
#일번지통닭
#일번지통닭박박사
#치킨박박사
#통닭박박사
#강남맛집
#압구정맛집
#압구정로데오맛집
#닭맛집
#인생닭집
#치킨맛집
#인생맛집
#인생맛집박박사
#맛집추천박박사
#맛스타그램
#먹스타그램
#닭스타그램
#Foodie
#Food
#Yummy
#Eat
#Chicken
#만수동박박사_압구정
#만수동박박사_닭
#만수동박박사_치킨
#만수동박박사
#쿠쿡
#CooCook
#웅슐랭
#강남맛집
#강남역맛집
#강남가볼만한곳
#강남역파스타
#강남핫플
#강남술집
#청담맛집
#양재맛집
#논현맛집
#맛스타그램
#맛집
#인스타광고
#강남맛집
#거자필반
#삼성역맛집
#삼성역
#삼성역맛집추천
#삼성맛집
#삼성맛집추천
#강남맛집
#강남맛집추천
#강남맛집베스트
#강남맛집탐방
#국밥
#국밥맛집
#국밥충
#곰탕
#곰탕맛집

# Chap 6-1. 연습문제
인스타그램에 자동으로 로그인 한 후 “강남맛집추천”으로 검색하여 나오는 게시물 중 50개를
추출하여 사진과 해시태그를 수집하여 사진은 사진 폴더에 별도로 저장하고(사진파일의 이름은
1.jpg 부터 50.jpg) 해시태그는 txt파일 형식으로 저장하는 웹 크롤러를 저장하세요. (단 게시물의
사진이 여러 장 일 경우 대표사진 1 장만 수집하여 저장하면 됩니다)

In [ ]:
##########################################################################
# 인스타 그램의 해쉬태그 수집하기 - by 서진수
##########################################################################
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import os
import random
import unicodedata   # 인스타그램의 해시태그 수집 중 자음/모음 분리현상 방지용 모듈

#Step 2. 사용자에게 필요한 정보들을를 입력 받습니다.
print("=" *70)
print("   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다")
print("   본 제품은 서진수가 교육용으로 특별 제작했으며 ")
print("   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다")
print("   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^")
print("=" *70)

v_id = input("1.인스타그램의 ID를 입력하세요: ")
v_passwd = input("2.인스타그램의 비밀번호를 입력하세요: ")
query_txt = input("3.검색할 해쉬태그를 입력하세요(예: 강남맛집): ")
cnt = int(input('4.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 10)

f_dir=input('5.파일이 저장될 경로만 쓰세요(기본경로 : c:\\temp\\ ) : ')
if f_dir =='' :
    f_dir = "c:\\temp\\"

#Step 3. 결과를 저장할 폴더명과 파일명을 설정하고 폴더를 생성합니다.
s_time = time.time( )
query_txt2 = '인스타그램'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

# Step 4. 인스타그램 자동 로그인 하기
chrome_path = "c:/temp/chromedriver_91/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.instagram.com/")
driver.maximize_window()
time.sleep(random.randrange(1,5))

print("\n")
print("요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^")
print("\n")

#ID와 비번 입력후 로그인하기
eid = driver.find_element_by_name('username')
for a in v_id :
        eid.send_keys(a)
        time.sleep(0.3)
epwd = driver.find_element_by_name('password')
for b in v_passwd :
        epwd.send_keys(b)
        time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(5)

# Step 5. 검색할 해쉬태그 입력하기
#로그인 정보 나중에 저장하기
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()

time.sleep(1)

# 검색할 키워드 입력하기
element = driver.find_element_by_xpath('''//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input''')

for c in query_txt :
    element.send_keys(c)
    time.sleep(0.2)

time.sleep(5)
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a').click()

import pandas as pd

# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(10)

print('요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^')
item=[]     # 인스타그램 URL 주소 저장할 리스트
item2=[]
    
a = 1
while (a <= real_cnt):
    scroll_down(driver) 

    # Step 6. 전체 게시물의 원본 URL 추출하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    all = soup.find('article','KC1QD').find_all('a')

    for i in all:    
        url = i['href']
        item.append(url)          
        item2 = pd.Series(item).drop_duplicates()
    
        if len(item2) >= cnt :
            break
    a += 1
    print('요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^')
    
    if len(item2) >= cnt :
        break
   
# 추출된 URL 사용하여 전체 URL 완성하기
full_url=[]
url_cnt = 0
for x in range(0,len(item2)) :
    url = 'https://www.instagram.com' +item[x]
    full_url.append(url)
    url_cnt += 1
    
    if url_cnt > cnt:
        break

# 아래 for 반복문은 수집된 URL 주소를 확인하는 코드임        
#for c in range(1,len(full_url)+1) :
#    print(c,':',full_url[c-1])

#Step 7. 각 페이지별로 그림과 해쉬태그를 수집하기
count = 1        # 추출 데이터 건수 세기
hash_txt = []    # 해쉬 태그 저장 

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
import sys
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

count = 0

os.makedirs(f_dir+s+'-'+query_txt2+'-'+query_txt)
f_name=f_dir+s+'-'+query_txt2+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'

for c in range(0,len(full_url)) :
    driver.get(full_url[c])
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    f = open(f_name, 'a',encoding='UTF-8')

    # 해당 페이지의 해시태그 수집
    tags = soup.find('div','EtaWk')

    try :
        tags_1 = tags.find_all('a')
    except :
        pass
    else :
        for d in range(0, len(tags_1)) :
            tags = tags_1[d].get_text()
            tags_11 = tags.translate(bmp_map)
            tags_2 = unicodedata.normalize('NFC', tags_11)

            for i in tags_2 :
                if i[0:1]=='#' :
                    hash_txt.append(tags_2)
                    print(tags_2)
                    f.write("\n" + str(tags_2))
    f.close()
    count += 1
    
#Step 7. 요약 정보 출력하기    
e_time = time.time( )
t_time = e_time - s_time

print("=" *100)
print("총 소요시간: %s 초" %round(t_time,1))
print("총 저장 건수: %s 건 " %count)
print("파일 저장 경로: %s" %f_name)
print("=" *100)

driver.close( )

In [47]:
#Chap 7-1. 이미지 다운로드용 웹크롤러 만들기
# Step 1. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import urllib
import time
import sys
import re
import math
import os
import random

f_dir = input('사진을 저장할 폴더를 지정하세요(예: c:\data\) : ')
query_txt ='사진저장'

#Step 2. 파일을 저장할 폴더를 생성합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.chdir(f_dir)
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)
f_result_dir = f_dir+s+'-'+query_txt

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
path = "c:/temp/chromedriver_91/chromedriver.exe"
driver = webdriver.Chrome(path)

s_time = time.time( )      # 크롤링 시작 시간을 위한 타임 스탬프를 찍습니다

driver.get("https://korean.visitkorea.or.kr/detail/rem_detail.html?cotid=be3db10c-b642-409c-81cc-c4cdecb5bd8b&temp=")
time.sleep(2)  # 페이지가 모두 열릴 때 까지 2초 기다립니다.

# 학습목표 1: 자동 스크롤다운 함수 만들기
# Step 4.  스크롤다운 함수를 생성한 후 실행합니다.
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)
    
scroll_down(driver) 
    

# 본문의 사진 정보를 가져와서 지정된 폴더에 저장하기    
# Step 5. 이미지 추출하여 저장하기 

file_no = 0                                
count = 1
img_src2=[]

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
img_src = soup.find('div','box_txtPhoto').find_all('img')

for i in img_src :
        img_src1=i['src']
        img_src2.append(img_src1)
        count += 1

for i in range(0,len(img_src2)) :
        try :
                urllib.request.urlretrieve(img_src2[i],str(file_no)+'.jpg')
        except :
                continue        
        file_no += 1                
        time.sleep(0.5)      
        print("%s 번째 이미지 저장중입니다=======" %file_no)
       
# Step 6. 요약 정보를 출력합니다                
e_time = time.time( )
t_time = e_time - s_time

store_cnt = file_no -1

print("=" *70)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("총 저장 건수는 %s 건 입니다 " %file_no)
print("파일 저장 경로: %s 입니다" %f_result_dir)
print("=" *70)

driver.close( )

사진을 저장할 폴더를 지정하세요(예: c:\data\) : c:\data\
1 번째 이미지 저장중입니다=======
2 번째 이미지 저장중입니다=======
3 번째 이미지 저장중입니다=======
4 번째 이미지 저장중입니다=======
5 번째 이미지 저장중입니다=======
6 번째 이미지 저장중입니다=======
7 번째 이미지 저장중입니다=======
8 번째 이미지 저장중입니다=======
9 번째 이미지 저장중입니다=======
10 번째 이미지 저장중입니다=======
11 번째 이미지 저장중입니다=======
12 번째 이미지 저장중입니다=======
13 번째 이미지 저장중입니다=======
14 번째 이미지 저장중입니다=======
총 소요시간은 20.6 초 입니다 
총 저장 건수는 14 건 입니다 
파일 저장 경로: c:\data\2021-07-14-14-23-08-사진저장 입니다


# Chap 7. 다양한 이미지 수집 크롤러 만들기 

In [38]:
#Chap 7-1. 이미지 다운로드용 웹크롤러 만들기
# Step 1. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import urllib
import time
import sys
import re
import math
import os
import random

f_dir = input('사진을 저장할 폴더를 지정하세요(예: c:\data\) : ')
query_txt ='사진저장'

#Step 2. 파일을 저장할 폴더를 생성합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.chdir(f_dir)
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)
f_result_dir = f_dir+s+'-'+query_txt

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
path = "c:/temp/chromedriver_91/chromedriver.exe"
driver = webdriver.Chrome(path)

s_time = time.time( )      # 크롤링 시작 시간을 위한 타임 스탬프를 찍습니다

driver.get("https://korean.visitkorea.or.kr/detail/rem_detail.html?cotid=be3db10c-b642-409c-81cc-c4cdecb5bd8b&temp=")
time.sleep(2)  # 페이지가 모두 열릴 때 까지 2초 기다립니다.

# 학습목표 1: 자동 스크롤다운 함수 만들기
# Step 4.  스크롤다운 함수를 생성한 후 실행합니다.
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)
    
scroll_down(driver) 
    

# 본문의 사진 정보를 가져와서 지정된 폴더에 저장하기    
# Step 5. 이미지 추출하여 저장하기 

file_no = 0                                
count = 1
img_src2=[]

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
img_src = soup.find('div','box_txtPhoto').find_all('img')

for i in img_src :
        img_src1=i['src']
        img_src2.append(img_src1)
        count += 1

for i in range(0,len(img_src2)) :
        try :
                urllib.request.urlretrieve(img_src2[i],str(file_no)+'.jpg')
        except :
                continue        
        file_no += 1                
        time.sleep(0.5)      
        print("%s 번째 이미지 저장중입니다=======" %file_no)
       
# Step 6. 요약 정보를 출력합니다                
e_time = time.time( )
t_time = e_time - s_time

store_cnt = file_no -1

print("=" *70)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("총 저장 건수는 %s 건 입니다 " %file_no)
print("파일 저장 경로: %s 입니다" %f_result_dir)
print("=" *70)

driver.close( )

사진을 저장할 폴더를 지정하세요(예: c:\data\) : c:\data\
1 번째 이미지 저장중입니다=======
2 번째 이미지 저장중입니다=======
3 번째 이미지 저장중입니다=======
4 번째 이미지 저장중입니다=======
5 번째 이미지 저장중입니다=======
6 번째 이미지 저장중입니다=======
7 번째 이미지 저장중입니다=======
8 번째 이미지 저장중입니다=======
9 번째 이미지 저장중입니다=======
10 번째 이미지 저장중입니다=======
11 번째 이미지 저장중입니다=======
12 번째 이미지 저장중입니다=======
13 번째 이미지 저장중입니다=======
14 번째 이미지 저장중입니다=======
총 소요시간은 53.1 초 입니다 
총 저장 건수는 14 건 입니다 
파일 저장 경로: c:\data\2021-07-14-13-58-03-사진저장 입니다


In [50]:
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd    
import os
import math


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print(" 쿠팡 사이트의 식품 카테고리 Best Seller 상품 정보 추출하기 ")
print("=" *80)

cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt/60)

f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본경로:c:\\temp\\):")
if f_dir == '' :
    f_dir = "c:\\temp\\"
    
print("\n")

if cnt > 30 :
      print("    요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")

#Step 3.저장될 파일 경로와 이름을 지정합니다
sec_name = '식품'
query_txt='쿠팡'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s+'-'+query_txt+'-'+sec_name)

ff_dir=f_dir+s+'-'+query_txt+'-'+sec_name
ff_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.txt'
fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xls'

# 제품 이미지 저장용 폴더 생성
img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)
    
s_time = time.time( )

#Step 4. 웹사이트 접속 후 해당 메뉴로 이동합니다.
chrome_path = "c:/temp/chromedriver_91/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
query_url='https://www.coupang.com/'
driver.get(query_url)
time.sleep(5)

# 쿠키 지우기
driver.delete_all_cookies()
time.sleep(2)

# 분야별 더보기 버튼을 눌러 페이지를 엽니다
driver.find_element_by_xpath("""//*[@id="header"]/div""").click( )
driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[4]/a""").click( )

#Step 5. 내용을 수집합니다
print("\n")
print("===== 곧 수집된 결과를 출력합니다 ^^ ===== ")
print("\n")

ranking2=[]        #제품의 판매순위 저장
title2=[]          #제품 정보 저장
p_price2=[]        #현재 판매가 저장
discount2 = []     #할인율 저장
sat_count2=[]      #상품평 수 저장

img_src2=[]   # 이미지 URL 저장변수
file_no = 0   # 이미지 파일 저장할 때 번호
count = 1     # 총 게시물 건수 카운트 변수

def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)

scroll_down(driver)   #현재화면의 가장 아래로 스크롤다운합니다

for x in range(1,page_cnt + 1) :
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    item_result = soup.find('ul','baby-product-list').find_all('li')

    for li in item_result :
        if cnt < count :
            break

        # 제품 이미지 다운로드 하기
        import urllib.request
        import urllib

        try :
            photo = li.find('dt','image').find('img')['src']
        except AttributeError : # 이미지가 없을 경우도 있기 때문 
            continue

        file_no += 1
        full_photo = 'https:' + photo
        urllib.request.urlretrieve(full_photo,str(file_no)+'.jpg')
        time.sleep(0.5)

        #제품 내용 추출하기
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("-----------------------------------------------------"+"\n")
        print("-" *70)

        ranking = count
        print("1.판매순위:",ranking)
        f.write('1.판매순위:'+ str(ranking) + "\n")

        try :
            t = li.find('div','name').get_text().replace("\n","")
        except :
            title = '제품소개가 없습니다'
            print(title.replace("\n",""))
            f.write('2.제품소개:'+ title + "\n")
        else :
            title = t.replace("\n","").strip()
            print("2.제품소개:", title.replace("\n","").strip())                  
            f.write('2.제품소개:'+ title + "\n")

        try :
            p_price = li.find('strong','price-value').get_text().replace("\n","")
        except :
            p_price = '0'
            print("3.판매가격:", p_price.replace("\n",""))
            f.write('3.판매가격:'+ p_price + "\n")
        else :
            print("3.판매가격:", p_price.replace("\n",""))
            f.write('3.판매가격:'+ p_price + "\n")

        try :
            discount = li.find('span','discount-percentage').get_text().replace("\n","")
        except  :
            discount = '0'
            print("4:할인률:", discount)
            f.write('4.할인율:'+ discount + "\n")
        else :
            print("4:할인률:", discount)
            f.write('4.할인율:'+ discount + "\n")

        try :
            sat_count_1 = li.find('span','rating-total-count').get_text()
            sat_count_2 = sat_count_1.replace("(","").replace(")","")
        except  :
            sat_count_2='0'
            print('5.상품평 수: ',sat_count_2)
            f.write('5.상품평 수:'+ sat_count_2 + "\n")
        else :
            print('5.상품평 수:',sat_count_2)
            f.write('5.상품평 수:'+ sat_count_2 + "\n")

        print("-" *70)

        f.close( )             
        time.sleep(0.5)

        ranking2.append(ranking)
        title2.append(title.replace("\n",""))

        p_price2.append(p_price.replace("\n",""))
        discount2.append(discount)

        try :   
            sat_count2.append(sat_count_2)
        except IndexError :
            sat_count2.append(0)

        count += 1
    x += 1                
    driver.find_element_by_link_text('%s' %x).click() # 다음 페이지번호 클릭
          
#step 6. csv , xls 형태로 저장하기              
co_best_seller = pd.DataFrame()
co_best_seller['판매순위']=ranking2
co_best_seller['제품소개']=pd.Series(title2)
co_best_seller['제품판매가']=pd.Series(p_price2)
co_best_seller['할인율']=pd.Series(discount2)
co_best_seller['상품평수']=pd.Series(sat_count2)

# csv 형태로 저장하기
co_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
co_best_seller.to_excel(fx_name ,index=False)

e_time = time.time( )
t_time = e_time - s_time

count -= 1
print("\n")
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

#Step 7. xls 파일에 제품 이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api                   #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120   

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

# 이름 불러오기
for a in range(2,cnt+2) :
      col_name='C'+str(a)
      col_name2.append(col_name)
      time.sleep(0,3)

# 사진 불러오기
for b in range(1,cnt+1) :
      file_name=img_dir+'\\'+str(b)+'.jpg'
      file_name2.append(file_name)
      time.sleep(0,3)

# 합치는 과정    
for i in range(0,cnt) :
      rng = ws.Range(col_name2[i])
      image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
      excel.Visible=True
      excel.ActiveWorkbook.Save()

driver.close( )

 쿠팡 사이트의 식품 카테고리 Best Seller 상품 정보 추출하기 
1.크롤링 할 건수는 몇건입니까?: 10
2.파일을 저장할 폴더명만 쓰세요(기본경로:c:\temp\):c:\temp\


    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~


===== 곧 수집된 결과를 출력합니다 ^^ ===== 


----------------------------------------------------------------------
1.판매순위: 1
2.제품소개: 바다자리 개조개, 700g, 1개
3.판매가격: 15,800
4:할인률: 19%
5.상품평 수: 2
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 2
2.제품소개: 폴바셋 시그니처 블렌드 원두+리유저블 텀블러 증정, 200g
3.판매가격: 13,300
4:할인률: 12%
5.상품평 수: 38
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 3
2.제품소개: 제주 삼다수, 2L, 24개
3.판매가격: 23,490
4:할인률: 40%
5.상품평 수: 303272
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 4
2.제품소개: 가야산 천년수 무라벨, 2L, 24개
3.판매가격: 9,900
4:할인률: 0
5.상품평 수: 23485
----

WebDriverException: Message: chrome not reachable
  (Session info: chrome=91.0.4472.124)
